In [1]:
from  __future__ import absolute_import
from __future__ import print_function
from ImageDataGeneratorCustom import ImageDataGeneratorCustom
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.layers import *
from keras.models import Model, load_model
from keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras import layers
from keras import optimizers
import shutil
import random
from PIL import Image
import numpy as np
import logging
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras import layers
import tensorflow_hub as hub
import shutil
import random
from PIL import Image
import pickle
import shutil
from tensorflow import debugging
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import tensorflow as tf
from pathlib import Path
from keras import applications
from keras import layers
from keras import losses
from keras import optimizers
from keras import metrics
from keras import Model
from keras.applications import resnet

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)


print(tf.__version__)

2.8.0


In [2]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2"

feature_extractor = hub.KerasLayer(URL, input_shape=(224, 224,3),trainable=False)
# emb = Dense(1024, activation='linear')(feature_extractor)
# embedding = Lambda(lambda  x: K.l2_normalize(x,axis=1))(emb)


embedding = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Dense(1024, activation='linear', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    # tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Lambda(lambda  x: K.l2_normalize(x,axis=1))
])

class DistanceLayer(layers.Layer):
    """
    This layer is responsible for computing the distance between the anchor
    embedding and the positive embedding, and the anchor embedding and the
    negative embedding.
    """

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, anchor, positive, negative):
        ap_distance = tf.reduce_sum(tf.square(anchor - positive), -1)
        an_distance = tf.reduce_sum(tf.square(anchor - negative), -1)
        return (ap_distance, an_distance)


anchor_input = layers.Input(name="anchor", shape=(224, 224,3))
positive_input = layers.Input(name="positive", shape=(224, 224,3))
negative_input = layers.Input(name="negative", shape=(224, 224,3))

distances = DistanceLayer()(
    embedding(anchor_input),
    embedding(positive_input),
    embedding(negative_input),
)

model = Model(
    inputs=[anchor_input, positive_input, negative_input], outputs=distances
)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 anchor (InputLayer)            [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 positive (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 negative (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                            

In [3]:
class SiameseModel(Model):
    """The Siamese Network model with a custom training and testing loops.

    Computes the triplet loss using the three embeddings produced by the
    Siamese Network.

    The triplet loss is defined as:
       L(A, P, N) = max(‖f(A) - f(P)‖² - ‖f(A) - f(N)‖² + margin, 0)
    """

    def __init__(self, siamese_network, margin=0.5):
        super(SiameseModel, self).__init__()
        self.siamese_network = siamese_network
        self.margin = margin
        self.loss_tracker = metrics.Mean(name="loss")

    def call(self, inputs):
        return self.siamese_network(inputs)

    def train_step(self, data):
        # GradientTape is a context manager that records every operation that
        # you do inside. We are using it here to compute the loss so we can get
        # the gradients and apply them using the optimizer specified in
        # `compile()`.
        with tf.GradientTape() as tape:
            loss = self._compute_loss(data)

        # Storing the gradients of the loss function with respect to the
        # weights/parameters.
        gradients = tape.gradient(loss, self.siamese_network.trainable_weights)

        # Applying the gradients on the model using the specified optimizer
        self.optimizer.apply_gradients(
            zip(gradients, self.siamese_network.trainable_weights)
        )

        # Let's update and return the training loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def test_step(self, data):
        loss = self._compute_loss(data)

        # Let's update and return the loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def _compute_loss(self, data):
        # The output of the network is a tuple containing the distances
        # between the anchor and the positive example, and the anchor and
        # the negative example.
        ap_distance, an_distance = self.siamese_network(data)

        # Computing the Triplet Loss by subtracting both distances and
        # making sure we don't get a negative value.
        loss = ap_distance - an_distance
        loss = tf.maximum(loss + self.margin, 0.0)
        return loss

    @property
    def metrics(self):
        # We need to list our metrics here so the `reset_states()` can be
        # called automatically.
        return [self.loss_tracker]

In [4]:
siamese_model = SiameseModel(model)

# for layer in deep_rank_model.layers:
#     print (layer.name, layer.output_shape)

model_path = "./siamese_model"

# siamese_model.summary()

In [5]:
# tf.saved_model.load(deep_rank_model, "./deep_ranking")
siamese_model.load_weights(model_path)



In [6]:
imgs = 2
pred = model.predict((np.random.rand(imgs,224,224,3),np.random.rand(imgs,224,224,3),np.random.rand(imgs,224,224,3)))
# print(pred.shape)
print(pred)

(array([0.04324277, 0.03130914], dtype=float32), array([0.04379506, 0.04622034], dtype=float32))


# test set predictions

In [7]:
# dictionary with all the image names and their features
features_dict = {}

#images (warning this takes about 11GB of RAM)
images = np.zeros((10000,224,224,3))

# convert every image into features
line_id = 0
for filename in os.listdir('food_224'):
    img = np.array(Image.open('food_224/' + filename))/255
    images[line_id] = img
    line_id += 1
    if line_id % 10000 == 0:
        print(line_id)
        break
    
print(images.shape)



10000
(10000, 224, 224, 3)


In [8]:
size = 100

features = siamese_model.predict([images[0:size], images[0:size], images[0:size]])

for i in np.arange(size,10001-size,size):
    print("predicting",i)
    f = siamese_model.predict([images[i:i+size], images[i:i+size], images[i:i+size]])
    features = np.concatenate([features, f])


line_id = 0
for filename in os.listdir('food_224'):
    features_dict[filename.replace(".jpg","")] = features[line_id]
    line_id += 1
    if line_id % 10000 == 0:
        print(line_id)
        break




predicting 100
predicting 200
predicting 300
predicting 400
predicting 500


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:GPU:0 to /job:localhost/replica:0/task:0/device:CPU:0 in order to run TensorDataset: Dst tensor is not initialized. [Op:TensorDataset]

In [ ]:
print("saving features")

# save the features dictionary to a file using pickle
with open('features.pickle', 'wb') as handle:
    pickle.dump(features_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)


print(features_dict['00001'].shape)
# print(features_dict)

saving features
(2048,)


In [ ]:

features_dict = {}
# load the features dictionary from the file
with open('features.pickle', 'rb') as handle:
    features_dict = pickle.load(handle)
    
print(features_dict['00001'].shape)

(2048,)


In [ ]:

pred = []
line_id = 0
with open('test_triplets.txt', 'r') as f:
    # go through the lines
    for line in f:
        line = line.replace("\n", "")


        embedding1 = features_dict[line.split(' ')[0]]
        embedding2 = features_dict[line.split(' ')[1]]
        embedding3 = features_dict[line.split(' ')[2]]

        
        #compare distance between embeddings
        distance1 = np.sqrt(np.sum((embedding1 - embedding2)**2))
        distance2 = np.sqrt(np.sum((embedding1 - embedding3)**2))

        pred.append(1 if distance1 < distance2 else 0)

        line_id += 1
        if line_id % 1000 == 0:
            print(line_id)
            # break

pred = np.array(pred,dtype=int)
#save to file as int
np.savetxt("submission.txt", pred, delimiter="\n", fmt="%d")


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
